<a href="https://colab.research.google.com/github/Chameleon-company/MOP-Code/blob/esha-2025/artificial-intelligence/T1_2025/T1_2025_Health_Behavior/dietary_monitoring_nutrition_ai/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("✅ Colab notebook is working!")


✅ Colab notebook is working!


In [1]:
#downloading the Food-101 dataset
!wget --no-check-certificate https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar -xvzf food-101.tar.gz


Streaming output truncated to the last 5000 lines.
food-101/images/cannoli/695879.jpg
food-101/images/cannoli/1767088.jpg
food-101/images/cannoli/3201965.jpg
food-101/images/cannoli/3778102.jpg
food-101/images/cannoli/2674065.jpg
food-101/images/cannoli/1169899.jpg
food-101/images/cannoli/1378086.jpg
food-101/images/cannoli/803909.jpg
food-101/images/cannoli/1067240.jpg
food-101/images/cannoli/3002709.jpg
food-101/images/cannoli/2612632.jpg
food-101/images/cannoli/3627612.jpg
food-101/images/cannoli/1798703.jpg
food-101/images/cannoli/560632.jpg
food-101/images/cannoli/512468.jpg
food-101/images/cannoli/2331523.jpg
food-101/images/cannoli/3118302.jpg
food-101/images/cannoli/1499412.jpg
food-101/images/cannoli/1557459.jpg
food-101/images/cannoli/146566.jpg
food-101/images/cannoli/96922.jpg
food-101/images/cannoli/1699562.jpg
food-101/images/cannoli/730051.jpg
food-101/images/cannoli/2177977.jpg
food-101/images/cannoli/2767028.jpg
food-101/images/cannoli/6564.jpg
food-101/images/cannoli/

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
import os


In [4]:
#setting up paths
#Food-101 images folder path
data_dir = '/content/food-101/images'

#picking a smaller set of classes for faster training
selected_classes = os.listdir(data_dir)[:10]  #picking 10 food types for now
print("Using these classes:", selected_classes)


Using these classes: ['garlic_bread', 'filet_mignon', 'tiramisu', 'croque_madame', 'crab_cakes', 'frozen_yogurt', 'macaroni_and_cheese', 'cheesecake', 'fish_and_chips', 'nachos']


In [5]:
#image data generator

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    classes=selected_classes,
    class_mode='categorical'
)

val_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    classes=selected_classes,
    class_mode='categorical'
)


Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [6]:
#building efficientnetB0 model

base_model = EfficientNetB0(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False
)

base_model.trainable = False  # freeze for now

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(selected_classes), activation='softmax')
])


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
#compilation and training

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5  # try 3–5 to start
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 818s 3s/step - accuracy: 0.0924 - loss: 2.3276 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 752s 3s/step - accuracy: 0.0971 - loss: 2.3028 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 749s 3s/step - accuracy: 0.1001 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 855s 3s/step - accuracy: 0.1040 - loss: 2.3026 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 751s 3s/step - accuracy: 0.0881 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026


In [8]:
print("Using these classes:", selected_classes)


Using these classes: ['garlic_bread', 'filet_mignon', 'tiramisu', 'croque_madame', 'crab_cakes', 'frozen_yogurt', 'macaroni_and_cheese', 'cheesecake', 'fish_and_chips', 'nachos']


In [9]:
#unfreezing the EfficientNet base model
base_model.trainable = True


In [10]:
#recompiling the model with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
#training the now-unfrozen model
fine_tune_history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    verbose=1
)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 2662s 10s/step - accuracy: 0.1195 - loss: 2.5177 - val_accuracy: 0.0995 - val_loss: 2.3060
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 2639s 11s/step - accuracy: 0.4015 - loss: 1.8774 - val_accuracy: 0.1540 - val_loss: 2.2525
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 2560s 10s/step - accuracy: 0.6137 - loss: 1.4641 - val_accuracy: 0.6465 - val_loss: 1.2669
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 2548s 10s/step - accuracy: 0.6833 - loss: 1.1478 - val_accuracy: 0.4795 - val_loss: 1.5942
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 2579s 10s/step - accuracy: 0.7488 - loss: 0.8921 - val_accuracy: 0.7615 - val_loss: 0.7961


In [12]:
model.save('efficientnet_food_classifier_10classes_v1.h5')

In [13]:
from google.colab import files
files.download('efficientnet_food_classifier_10classes_v1.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>